Construct a pyomo model based on the following linear program:

$$
\begin{array}{ll} \min & 2 x_1 + 3 x_2\\ \mathrm{s.t.} & 3 x_1 + 4 x_2 \geq 1\\ & x_1, x_2 \geq 0 \end{array}
$$

<!-- $$
\begin{array}{lll} 
\min & \sum_{j=1}^n c_j x_j &\\ \mathrm{s.t.} & \sum_{j=1}^n a_{ij} x_j \geq b_i & \forall i = 1 \ldots m\\ & x_j \geq 0 & \forall j = 1 \ldots n 
\end{array}
$$ -->

# 1. IMPORT

In [1]:
import pyomo as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# 2. DEFINE THE MODEL

In [10]:
# define the model
model = ConcreteModel()

# step 1: define the decision variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# step 2: define the constraints
def constraint_rule1(model):
    return 3*model.x1 + 4*model.x2 >= 1
model.cons1 = Constraint(rule=constraint_rule1)

# step 3: define the objective function
def objective_rule(model):
    return 2*model.x1 + 3*model.x2

# 3. SOLVE THE MODEL

In [11]:
SolverFactory('glpk').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1.0
  Upper bound: 1.0
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 3
  Number of nonzeros: 2
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.011902809143066406
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------

In [5]:
SolverFactory('scip').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: optimal solution found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.0
  Gap: 0.0
  Primal bound: 0.0
  Dual bound: 0.0
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displ

In [16]:
print(model.x2())

0.25
